In [22]:
import spacy
import empath

In [6]:
nlp = spacy.load('en')

In [7]:
doc = nlp('this is a sentence. Actually, two sentences')

In [21]:
doc.print_tree()[0]

/usr/local/Cellar/python/3.6.4_4/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py:193: DeprecationWarning: [W003] Positional arguments to Doc.merge are deprecated. Instead, use the keyword arguments, for example tag=, lemma= or ent_type=.
  "__main__", mod_spec)


{'word': 'is',
 'lemma': 'be',
 'NE': '',
 'POS_fine': 'VBZ',
 'POS_coarse': 'VERB',
 'arc': 'ROOT',
 'modifiers': [{'word': 'this',
   'lemma': 'this',
   'NE': '',
   'POS_fine': 'DT',
   'POS_coarse': 'DET',
   'arc': 'nsubj',
   'modifiers': []},
  {'word': 'sentence',
   'lemma': 'sentence',
   'NE': '',
   'POS_fine': 'NN',
   'POS_coarse': 'NOUN',
   'arc': 'attr',
   'modifiers': [{'word': 'a',
     'lemma': 'a',
     'NE': '',
     'POS_fine': 'DT',
     'POS_coarse': 'DET',
     'arc': 'det',
     'modifiers': []}]},
  {'word': '.',
   'lemma': '.',
   'NE': '',
   'POS_fine': '.',
   'POS_coarse': 'PUNCT',
   'arc': 'punct',
   'modifiers': []}]}

# Empath
Lexicon substitute for the commercial LIWC.

In [23]:
lexicon = empath.Empath()

In [26]:
lexicon.analyze('he hit the other person', normalize=True)

{'help': 0.0,
 'office': 0.0,
 'dance': 0.0,
 'money': 0.0,
 'wedding': 0.0,
 'domestic_work': 0.0,
 'sleep': 0.0,
 'medical_emergency': 0.0,
 'cold': 0.0,
 'hate': 0.0,
 'cheerfulness': 0.0,
 'aggression': 0.0,
 'occupation': 0.0,
 'envy': 0.0,
 'anticipation': 0.0,
 'family': 0.0,
 'vacation': 0.0,
 'crime': 0.0,
 'attractive': 0.0,
 'masculine': 0.0,
 'prison': 0.0,
 'health': 0.0,
 'pride': 0.0,
 'dispute': 0.0,
 'nervousness': 0.0,
 'government': 0.0,
 'weakness': 0.0,
 'horror': 0.0,
 'swearing_terms': 0.0,
 'leisure': 0.0,
 'suffering': 0.0,
 'royalty': 0.0,
 'wealthy': 0.0,
 'tourism': 0.0,
 'furniture': 0.0,
 'school': 0.0,
 'magic': 0.0,
 'beach': 0.0,
 'journalism': 0.0,
 'morning': 0.0,
 'banking': 0.0,
 'social_media': 0.0,
 'exercise': 0.0,
 'night': 0.0,
 'kill': 0.0,
 'blue_collar_job': 0.0,
 'art': 0.0,
 'ridicule': 0.0,
 'play': 0.0,
 'computer': 0.0,
 'college': 0.0,
 'optimism': 0.0,
 'stealing': 0.0,
 'real_estate': 0.0,
 'home': 0.0,
 'divine': 0.0,
 'sexual': 0.0

# Positive/Negative words

Positive words in ASCII, negative ISO-8859. `;` indicates comments, one empty line between comments and start of words.

In [34]:
with open('./positive-words.txt') as f:
    for w in f:
        if w.startswith(';'):
            continue
        else: break
    positive_words = set(w.strip().lower() for w in f)
    
with open('./negative-words.txt', encoding='ISO-8859-1') as f:
    for w in f:
        if w.startswith(';'):
            continue
        else: break
    negative_words = set(w.strip().lower() for w in f)

In [36]:
negative_words

{'intolerance',
 'layoff',
 'irrelevant',
 'flat-out',
 'nasty',
 'qualm',
 'slashing',
 'imbalance',
 'shrivel',
 'idiot',
 'inflammatory',
 'contempt',
 'skepticism',
 'sugarcoated',
 'dissonance',
 'disgraced',
 'panicking',
 'shadowy',
 'ravage',
 'harried',
 'stingy',
 'grumble',
 'unwillingly',
 'hideous',
 'inept',
 'fractious',
 'sceptical',
 'unlicensed',
 'miscalculation',
 'buzzing',
 'chatter',
 'divisiveness',
 'flairs',
 'effrontery',
 'bump',
 'dirty',
 'dissatisfaction',
 'distort',
 'despondent',
 'ignoble',
 'extravagantly',
 'impunity',
 'killed',
 'ferocity',
 'coldly',
 'abort',
 'dismalness',
 'derogatory',
 'sober',
 'lure',
 'unstable',
 'angriness',
 'backache',
 'agony',
 'worsen',
 'profanity',
 'rupture',
 'wiles',
 'smack',
 'perfidious',
 'maladjusted',
 'brutality',
 'primitive',
 'disobey',
 'afflictive',
 'flagrantly',
 'inadequacy',
 'onslaught',
 'prejudge',
 'sluggish',
 'temper',
 'woefully',
 'tangle',
 'slumpping',
 'aloof',
 'warned',
 'teasingly

# Subjclues

Used vim to transfer them to JSON; removed 'polarity' and 'mpqpolarity' since only one entry uses these.

In [48]:
import pandas as pd
subjclues = pd.read_json('./subjclues.json')

In [49]:
subjclues

,len,pos1,priorpolarity,stemmed1,type,word1
0,1,adj,negative,n,weaksubj,abandoned
1,1,noun,negative,n,weaksubj,abandonment
2,1,verb,negative,y,weaksubj,abandon
3,1,verb,negative,y,strongsubj,abase
4,1,anypos,negative,y,strongsubj,abasement
5,1,verb,negative,y,strongsubj,abash
6,1,verb,negative,y,weaksubj,abate
7,1,verb,negative,y,weaksubj,abdicate
8,1,adj,negative,n,strongsubj,aberration
9,1,noun,negative,n,strongsubj,aberration
